In [1]:
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Activation, Flatten
import numpy as np
from google.colab import files
import pandas as pd
import io
import random

Using TensorFlow backend.


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# check auth
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
x_downloaded = drive.CreateFile({'id':'1VHQaaId7CpxrKp_8rYA_olEnCx9M8UbD'})
y_downloaded = drive.CreateFile({'id':'1awM8F7xjyasxjMRC17sf-bqWxUYFzwct'})
x_downloaded.GetContentFile('X64.npy')
y_downloaded.GetContentFile('Y64.npy')
x = np.load('X64.npy')
y = np.load('Y64.npy')
print(y)

[0 0 0 ... 1 1 1]


In [4]:
x_train = []
x_test = []
y_train = []
y_test = []
for num in range(len(x)):
  if random.randrange(10) == 0:
    x_test.append(x[num])
    y_test.append(y[num])
  else:
    x_train.append(x[num])
    y_train.append(y[num])
print(len(x_train))
print(len(x_test))
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

4908
542


In [0]:
print(type(x_train))
print(y_train)
x_train = x_train / 255
x_test = x_test / 255
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [0]:
print(len(x_train))
print(x_train.shape)
print(x_train)

In [7]:
model = Sequential()

model.add(Conv2D(64,(3,3),padding='same',input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(8,8)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=((8,8))))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 64, 64, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         73856     
__________

In [8]:
history = model.fit(x_train, y_train, batch_size=512, epochs=20, verbose=1, validation_split=0.1)

Train on 4417 samples, validate on 491 samples
Epoch 1/20
4417/4417 [==============================] - 10s 2ms/step - loss: 0.5582 - acc: 0.7351 - val_loss: 0.2610 - val_acc: 1.0000
Epoch 2/20
4417/4417 [==============================] - 3s 690us/step - loss: 0.4518 - acc: 0.7983 - val_loss: 0.0842 - val_acc: 1.0000
Epoch 3/20
4417/4417 [==============================] - 3s 684us/step - loss: 0.3764 - acc: 0.7983 - val_loss: 0.1129 - val_acc: 1.0000
Epoch 4/20
4417/4417 [==============================] - 3s 683us/step - loss: 0.2905 - acc: 0.8051 - val_loss: 0.0543 - val_acc: 1.0000
Epoch 5/20
4417/4417 [==============================] - 3s 685us/step - loss: 0.2269 - acc: 0.9235 - val_loss: 0.1029 - val_acc: 0.9715
Epoch 6/20
4417/4417 [==============================] - 3s 683us/step - loss: 0.1845 - acc: 0.9545 - val_loss: 0.0379 - val_acc: 0.9980
Epoch 7/20
4417/4417 [==============================] - 3s 681us/step - loss: 0.1486 - acc: 0.9758 - val_loss: 0.0434 - val_acc: 0.9674
Ep

In [9]:
score = model.evaluate(x_test, y_test)
print(score)

542/542 [==============================] - 0s 726us/step
[0.01608041311789737, 0.9944649446494465]


In [0]:
open('model.json', 'w').write(model.to_json())
model.save_weights('param.hdf5')

In [0]:
files.download('model.json')
files.download('param.hdf5')

In [0]:
a